# Apply Top-Department Cascade (Round 1) — EV/ICE Split + Rich Notes

This notebook performs one **greedy electrification round** and outputs **two datasets**:

- **Top Dept EVs**: only the vehicles in the top-ranked department that were **converted to EVs** in this round, with Notes indicating where their displaced ICE was cascaded (VIN + Dept).
- **Remaining Fleet ICE**: the rest of the fleet (all departments) **excluding** those EV-converted rows; for any vehicles that received an incoming ICE, their Notes include which VIN (and Dept) they **replaced**.

**Inputs (defaults):**
- Fleet: `filtered_fleet_noGTPD.csv`
- Cascades directory: `All_Electric_Cascades/`
- Savings summary: `Dept Savings Results/Department_Savings_Summary.csv`

**Outputs:**
- `fleet_round1_top_dept_EVs.csv`
- `fleet_round1_remaining_ICE.csv`
- `fleet_after_round1_change_log.csv` (diagnostics)


In [1]:

from pathlib import Path

# ---- Parameters ----
CURRENT_YEAR = 2025

FLEET_PATH   = Path("filtered_fleet_noGTPD.csv")
CASCADES_DIR = Path("All_Electric_Cascades")
SAVINGS_SUM  = Path("Dept Savings Results") / "Department_Savings_Summary.csv"

OUT_EVS = Path("fleet_round1_top_dept_EVs.csv")
OUT_ICE = Path("fleet_round1_remaining_ICE.csv")
OUT_LOG = Path("fleet_after_round1_change_log.csv")

# If you want to force a specific department, set this to a string; otherwise None to use top-ranked.
OVERRIDE_DEPARTMENT = None

# Replace ALL remaining vehicles in the selected department that have EV alternatives (not just Step-1s)?
ELECTRIFY_NON_CASCADE = True


In [2]:

import pandas as pd
import numpy as np
from typing import List, Dict, Any, Tuple, Optional

FUEL_COLS = ["Q1 Gallons","Q2 Gallons","6 Month Gallons","Annual Gallons (forecasted)","age * gallons"]
AGE_COLS  = ["Age_Years","Age_For_Calc"]

def ensure_notes_column(df: pd.DataFrame):
    if "Notes" not in df.columns:
        df["Notes"] = ""
    return df

def append_note(text: str, addition: str) -> str:
    text = str(text) if pd.notna(text) else ""
    addition = addition.strip()
    return (text + " | " + addition).strip(" |") if text else addition

def pick_top_department(summary_path: Path, override_dept: Optional[str]) -> str:
    if override_dept:
        return override_dept
    df = pd.read_csv(summary_path)
    if df.empty:
        raise ValueError("Savings summary is empty.")
    if "Department" not in df.columns or "Total 10-Year Savings ($)" not in df.columns:
        raise ValueError("Savings summary missing required columns: 'Department', 'Total 10-Year Savings ($)'.")
    df = df.sort_values("Total 10-Year Savings ($)", ascending=False)
    return str(df.iloc[0]["Department"]).strip()

def load_cascades_for_department(casc_dir: Path, department: str) -> list[dict]:
    rows = []
    for p in casc_dir.glob("*.csv"):
        if p.name.endswith("__log.csv"):
            continue
        try:
            cdf = pd.read_csv(p)
        except Exception:
            continue
        cdf.columns = cdf.columns.str.strip()
        if "Department" not in cdf.columns:
            dept_guess = p.stem.rsplit("_", 1)[0].replace("_", " ").strip()
            cdf["Department"] = dept_guess
        sub = cdf[cdf["Department"].astype(str).str.strip() == department].copy()
        if sub.empty:
            continue
        sub["__source_file__"] = p.name
        rows.extend(sub.to_dict("records"))
    return rows

def detect_steps(rec: dict) -> List[int]:
    steps = []
    i = 1
    while f"Step {i} Vehicle" in rec:
        steps.append(i); i += 1
    return steps

def convert_to_ev_inplace(df: pd.DataFrame, idx: int) -> None:
    """Convert a row to its EV alternative in-place and mark it."""
    ev_model = str(df.at[idx, "Electric Alternative"]).strip()
    if not ev_model or ev_model.lower() in {"nan","none"}:
        raise ValueError("No Electric Alternative")
    df.at[idx, "Vehicle Model"] = ev_model
    df.at[idx, "Model Year"] = CURRENT_YEAR
    for c in FUEL_COLS:
        if c in df.columns: df.at[idx, c] = 0
    for c in AGE_COLS:
        if c in df.columns: df.at[idx, c] = 0
    df.at[idx, "Notes"] = append_note(df.at[idx, "Notes"], f"Converted to EV {CURRENT_YEAR} (auto-step)")
    df.at[idx, "Is_EV_Converted"] = True  # flag for splitting outputs

def apply_single_cascade(df: pd.DataFrame, original_df: pd.DataFrame, rec: dict,
                         logs: list[dict], touched_vins: set[str]) -> None:
    steps = detect_steps(rec)
    if len(steps) < 2:
        return
    vins = [str(rec.get(f"Step {i} Vehicle","")).strip() for i in steps]
    vins = [v for v in vins if v]
    if len(vins) < 2:
        return

    idx_map = {str(v): i for i, v in enumerate(df["VIN"].astype(str))}
    missing = [v for v in vins if v not in idx_map]
    if missing:
        logs.append({"event":"skip_cascade_missing_vin","vins":missing,"source":rec.get("__source_file__")})
        return
    if any(v in touched_vins for v in vins):
        logs.append({"event":"skip_cascade_reused_vin","vins":[v for v in vins if v in touched_vins]})
        return

    # --- Step 1: convert to EV ---
    v1 = vins[0]; i1 = idx_map[v1]
    # Record the first target's department (before any overwrite) to annotate the EV note
    first_target = vins[1]
    dept_first_target = str(df.loc[idx_map[first_target], "Department"])

    try:
        convert_to_ev_inplace(df, i1)
        # Add note stating where its ICE was cascaded first
        df.at[i1, "Notes"] = append_note(df.at[i1, "Notes"], f"ICE cascaded to replace VIN {first_target} (Dept {dept_first_target})")
    except Exception as e:
        logs.append({"event":"skip_cascade_no_ev_alt","vin":v1,"err":str(e)})
        return

    # The displaced ICE payload is the pristine original of Step-1
    payload = original_df[original_df["VIN"].astype(str) == v1].iloc[0].copy()

    # --- Steps 2..k ---
    for j in range(1, len(vins)):
        v_target = vins[j]; i_target = idx_map[v_target]
        target_before = df.loc[i_target].copy()
        target_dept   = str(target_before.get("Department"))
        target_orig   = original_df[original_df["VIN"].astype(str) == v_target].iloc[0].copy()

        incoming = payload.copy()
        # Move to target's department
        incoming["Department"] = target_dept
        # Update Notes: include which VIN + Dept it replaced
        incoming_notes = append_note(incoming.get("Notes",""), f"Incoming ICE via cascade: replaced VIN {v_target} (Dept {target_dept})")
        incoming["Notes"] = incoming_notes
        # Ensure EV flag absent for incoming ICE
        incoming["Is_EV_Converted"] = False

        # Overwrite target with incoming ICE
        for col in df.columns:
            if col in incoming.index:
                df.at[i_target, col] = incoming[col]

        # Next payload is the original of the target
        payload = target_orig
        touched_vins.add(v_target)

    # Retire the final VIN row
    v_last = vins[-1]
    if v_last in idx_map:
        i_last = idx_map[v_last]
        logs.append({"event":"retire_final_vin","vin":v_last,"dept":str(df.loc[i_last,"Department"])})
        df.drop(index=i_last, inplace=True)
        df.reset_index(drop=True, inplace=True)

    touched_vins.update(vins)
    logs.append({"event":"applied_cascade","chain_vins":vins,"source":rec.get("__source_file__"),
                 "model":rec.get("Model"),"department_seed":rec.get("Department")})

def electrify_remaining_in_dept(df: pd.DataFrame, dept: str, casc_step1_vins: set[str], logs: list[dict]) -> None:
    mask = (df["Department"].astype(str).str.strip() == str(dept).strip())
    for idx in df[mask].index:
        vin = str(df.loc[idx,"VIN"]).strip()
        if vin in casc_step1_vins:
            continue
        ev_alt = str(df.loc[idx].get("Electric Alternative","")).strip()
        if ev_alt and ev_alt.lower() not in {"nan","none"}:
            try:
                convert_to_ev_inplace(df, idx)
                logs.append({"event":"electrified_no_cascade","vin":vin,"department":dept})
            except Exception as e:
                logs.append({"event":"skip_electrify_no_cascade","vin":vin,"department":dept,"err":str(e)})


In [3]:

# --- Run the round ---
from IPython.display import display

# Sanity checks
if not FLEET_PATH.exists():
    raise FileNotFoundError(f"Fleet file not found: {FLEET_PATH}")
if not SAVINGS_SUM.exists():
    raise FileNotFoundError(f"Savings summary not found: {SAVINGS_SUM}. Run Cost Modeller first.")
if not CASCADES_DIR.exists():
    raise FileNotFoundError(f"Cascades folder not found: {CASCADES_DIR}. Run Cascade Generator first.")

fleet_df = pd.read_csv(FLEET_PATH)
fleet_df = ensure_notes_column(fleet_df)
# init EV flag
if "Is_EV_Converted" not in fleet_df.columns:
    fleet_df["Is_EV_Converted"] = False

required = ["VIN","Department","Vehicle Model","Model Year","Electric Alternative","age * gallons"]
for c in required:
    if c not in fleet_df.columns:
        raise ValueError(f"Fleet file missing required column: {c}")

dept = pick_top_department(SAVINGS_SUM, OVERRIDE_DEPARTMENT)
print("Selected Department:", dept)

original_df = fleet_df.copy()
casc_rows = load_cascades_for_department(CASCADES_DIR, dept)
if not casc_rows:
    raise RuntimeError(f"No cascades found for '{dept}' in {CASCADES_DIR}.")

# Step-1 VINs (avoid double-electrifying later)
casc_step1_vins = { str(r.get("Step 1 Vehicle","")).strip() for r in casc_rows if str(r.get("Step 1 Vehicle","")).strip() }

logs: list[dict] = []
touched_vins: set[str] = set()

# Apply cascades (only those whose Step-1 is still in this dept)
for rec in casc_rows:
    v1 = str(rec.get("Step 1 Vehicle","")).strip()
    if not v1:
        continue
    idxs = fleet_df.index[fleet_df["VIN"].astype(str) == v1].tolist()
    if not idxs:
        logs.append({"event":"skip_cascade_missing_v1","vin":v1}); continue
    if str(fleet_df.loc[idxs[0],"Department"]).strip() != str(dept).strip():
        logs.append({"event":"skip_cascade_v1_not_in_dept_anymore","vin":v1,"expected_dept":dept}); continue
    apply_single_cascade(fleet_df, original_df, rec, logs, touched_vins)

# Electrify remaining EV-eligible vehicles in the dept (no cascade payload propagation)
if ELECTRIFY_NON_CASCADE:
    electrify_remaining_in_dept(fleet_df, dept, casc_step1_vins, logs)

# --- Split outputs ---
# EVs in the top department (converted this round)
ev_mask = (fleet_df["Department"].astype(str).str.strip() == dept) & (fleet_df["Is_EV_Converted"] == True)
top_dept_evs = fleet_df[ev_mask].copy()

# Remaining ICE vehicles (entire fleet excluding those EV rows)
remaining_ice = fleet_df[~ev_mask].copy()

# Save outputs
OUT_EVS.parent.mkdir(parents=True, exist_ok=True)
top_dept_evs.to_csv(OUT_EVS, index=False)
print("Wrote Top Dept EVs to:", OUT_EVS)

OUT_ICE.parent.mkdir(parents=True, exist_ok=True)
remaining_ice.to_csv(OUT_ICE, index=False)
print("Wrote Remaining ICE fleet to:", OUT_ICE)

OUT_LOG.parent.mkdir(parents=True, exist_ok=True)
pd.DataFrame(logs).to_csv(OUT_LOG, index=False)
print("Wrote change log to:", OUT_LOG)

# Quick peeks
print("\nTop Dept EVs preview:")
display(top_dept_evs.head(10))

print("\nRemaining ICE preview:")
display(remaining_ice.head(10))


Selected Department: Gtri
Wrote Top Dept EVs to: fleet_round1_top_dept_EVs.csv
Wrote Remaining ICE fleet to: fleet_round1_remaining_ICE.csv
Wrote change log to: fleet_after_round1_change_log.csv

Top Dept EVs preview:


,Unit,Vehicle Number,VIN,Model Year,Vehicle Model,Cost Per Mile,TXN(s),Q1 Gallons,Q2 Gallons,Q2 Spend,...,6 Month Gallons,Annual Gallons (forecasted),age * gallons,Electric Alternative,Replacement Vehicle,Unnamed: 17,Notes,Age_Years,Age_For_Calc,Is_EV_Converted
6,503,501,1B7FL26X2XS243387,2025,Yes,$0.33,1,0.0,0.0,$27.44,...,0.0,0.0,0.0,Yes,Ford F-150 Lightening,NaN,Converted to EV 2025 (auto-step),0,0,True
7,503,276,1FMCA11U6TZC05741,2025,Yes,$0.30,4,0.0,0.0,$139.33,...,0.0,0.0,0.0,Yes,Ford E-Transit,NaN,Converted to EV 2025 (auto-step),0,0,True
8,503,198,1FBNE3BL0CDB14274,2025,Yes,$0.27,19,0.0,0.0,$941.42,...,0.0,0.0,0.0,Yes,Ford E-Transit,NaN,Converted to EV 2025 (auto-step) | ICE cascade...,0,0,True
18,503,763,1FDSE35L59DA64633,2025,Yes,$0.00,1,0.0,0.0,$66.17,...,0.0,0.0,0.0,Yes,Ford E-Transit,NaN,Converted to EV 2025 (auto-step) | ICE cascade...,0,0,True
24,503,1034,1FM5K7B85GGB07005,2025,Yes,$0.14,5,0.0,0.0,$118.98,...,0.0,0.0,0.0,Yes,Ford Explorer EV,NaN,Converted to EV 2025 (auto-step) | ICE cascade...,0,0,True
25,503,150,1FM5K7B8XEGA18155,2025,Yes,$0.23,2,0.0,0.0,$78.28,...,0.0,0.0,0.0,Yes,Ford Explorer EV,NaN,Converted to EV 2025 (auto-step),0,0,True
26,503,1035,1FM5K7B83GGB07004,2025,Yes,$0.09,2,0.0,0.0,$73.75,...,0.0,0.0,0.0,Yes,Ford Explorer EV,NaN,Converted to EV 2025 (auto-step) | ICE cascade...,0,0,True
27,503,241,1FM5K7B87DGB15893,2025,Yes,$0.19,3,0.0,0.0,$67.79,...,0.0,0.0,0.0,Yes,Ford Explorer EV,NaN,Converted to EV 2025 (auto-step),0,0,True
29,503,948,1FM5K7B85FGA41585,2025,Yes,$0.23,1,0.0,0.0,$40.35,...,0.0,0.0,0.0,Yes,Ford Explorer EV,NaN,Converted to EV 2025 (auto-step),0,0,True
30,503,1630,1FMSK8BH9PGA00922,2025,Yes,$0.27,1,0.0,0.0,$8.07,...,0.0,0.0,0.0,Yes,Ford Explorer EV,NaN,Converted to EV 2025 (auto-step) | ICE cascade...,0,0,True



Remaining ICE preview:


,Unit,Vehicle Number,VIN,Model Year,Vehicle Model,Cost Per Mile,TXN(s),Q1 Gallons,Q2 Gallons,Q2 Spend,...,6 Month Gallons,Annual Gallons (forecasted),age * gallons,Electric Alternative,Replacement Vehicle,Unnamed: 17,Notes,Age_Years,Age_For_Calc,Is_EV_Converted
0,503,1249,3GNKBJRS3KS559526,2019,Chevrolet Blazer,$0.24,3,7.56,9.64,$31.83,...,17.20,34.40,206.40,Yes,Chevrolet Blazer EV,NaN,NaN,6,6,False
1,503,129,1GAHG39R821193893,2002,Chevrolet Express,$0.38,8,136.00,185.60,$499.26,...,321.60,643.20,14793.60,Yes,Ford E-Transit,NaN,NaN,23,23,False
2,503,147,2GCEC19W511210738,2001,Chevrolet Silverado 1500,$1.23,3,18.60,59.00,$158.71,...,77.60,155.20,3724.80,Yes,2024 Chevrolet Siverado EV,NaN,NaN,24,24,False
3,503,1647,3GCNAAED7RG413059,2024,Chevrolet Silverado 1500,$0.19,2,20.10,44.50,$119.71,...,64.60,129.20,129.20,Yes,2024 Chevrolet Siverado EV,NaN,NaN,1,1,False
4,503,437,3GCEC14Z27G175500,2007,Chevrolet Silverado Classic 1500,$0.29,3,60.60,45.50,$122.40,...,106.10,212.20,3819.60,Yes,2024 Chevrolet Siverado EV,NaN,NaN,18,18,False
5,503,245,2A8HR54159R505536,2009,Chrysler Town & Country,$0.26,2,16.30,13.00,$34.97,...,29.30,58.60,937.60,Yes,Isuzu NRR EV,NaN,NaN,16,16,False
9,503,608,1FBNE3BL2DDA49445,2013,Ford Econoline,$0.19,9,115.67,155.21,$466.75,...,270.88,541.76,6501.12,Yes,Ford E-Transit,NaN,NaN,12,12,False
10,503,1279,1FDWE3FK9NDC25048,2022,Ford Econoline,$0.63,6,77.80,164.80,$443.30,...,242.60,485.20,1455.60,Yes,Ford E-Transit,NaN,NaN,3,3,False
11,503,525,1FTSE34S67DB07130,2007,Ford Econoline,$1.02,5,110.90,124.00,$333.56,...,234.90,469.80,8456.40,Yes,Ford E-Transit,NaN,NaN,18,18,False
12,503,1278,1FDWE3FK7NDC25047,2022,Ford Econoline,$0.77,4,247.00,111.20,$299.12,...,358.20,716.40,2149.20,Yes,Ford E-Transit,NaN,NaN,3,3,False


In [4]:
# Diagnostics: show the last 20 log entries (if any)
try:
    log_df = pd.read_csv(OUT_LOG)
    display(log_df.tail(20))
except Exception as e:
    print("No log yet or failed to read log:", e)

,event,vin,dept,chain_vins,source,model,department_seed,vins,department
11,retire_final_vin,1FBZX2ZM6GKB25074,I&S (Bldg Services),NaN,NaN,NaN,NaN,NaN,NaN
12,applied_cascade,NaN,NaN,"['1FBZX2YM6KKB47506', '1FBZX2ZM6GKB25074']",Gtri_Ford Transit.csv,Ford Transit,Gtri,NaN,NaN
13,retire_final_vin,1FMZK1ZM2FKA65244,Housing,NaN,NaN,NaN,NaN,NaN,NaN
14,applied_cascade,NaN,NaN,"['1FTYE1YM0KKB31468', '1FMZK1ZM2FKA65244']",Gtri_Ford Transit.csv,Ford Transit,Gtri,NaN,NaN
15,retire_final_vin,1FDXE45FX3HB41339,Gtri,NaN,NaN,NaN,NaN,NaN,NaN
16,applied_cascade,NaN,NaN,"['1FBNE3BL0CDB14274', '1FDXE45FX3HB41339']",Gtri_Ford Econoline.csv,Ford Econoline,Gtri,NaN,NaN
17,retire_final_vin,1FTEE14Y3RHC13598,Gtri,NaN,NaN,NaN,NaN,NaN,NaN
18,applied_cascade,NaN,NaN,"['1FDSE35L59DA64633', '1FTEE14Y3RHC13598']",Gtri_Ford Econoline.csv,Ford Econoline,Gtri,NaN,NaN
19,retire_final_vin,1FM5K7B83DGA26600,Gtri,NaN,NaN,NaN,NaN,NaN,NaN
20,applied_cascade,NaN,NaN,"['1FM5K7B85GGB07005', '1FM5K7B83DGA26600']",Gtri_Ford Explorer.csv,Ford Explorer,Gtri,NaN,NaN
